In [1]:
import tensorflow as tf
tf.python.control_flow_ops = tf

import os
import pandas as pd
import numpy as np
import cv2
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout, \
MaxPooling2D, Conv2D, Lambda, Cropping2D, Convolution2D,\
AveragePooling2D
from keras.callbacks import History,TensorBoard, EarlyStopping, ModelCheckpoint
from sklearn.cross_validation import train_test_split

from copy import deepcopy
from sklearn.utils import shuffle


from generator import generator
from get_images import get_images


Using TensorFlow backend.
/root/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
datadirs=['/notebooks/udacity/new_training/map1_backward/',
                 '/notebooks/udacity/new_training/map1_forward/',
                 '/notebooks/udacity/new_training/map1_recovery_backward/',
                 '/notebooks/udacity/new_training/map1_recovery_forward/',
                 '/notebooks/udacity/new_training/map2_forward/',
                 '/notebooks/udacity/new_training/map2_backward/',
                 '/notebooks/udacity/new_training/map2_recovery_forward/',
                 '/notebooks/udacity/new_training/map2_recovery_backward/']

images=get_images(datadirs)

In [3]:
size=(40,80)

def preprocessing_resize(im):
    return cv2.resize(cv2.cvtColor(im, cv2.COLOR_BGR2YUV),(size[1],size[0]))

In [4]:
from car_models import *

In [5]:
grid=[#{'get_model':modified_lenet, 'correction':0.2, \
     #  'preprocessing':preprocessing_resize,'optimizer':'adam','gpu':'0','name':'lenet_02_adam'},\
     {'get_model':nvidia_net, 'correction':0.2, \
       'preprocessing':preprocessing_resize,'optimizer':'adam','gpu':'0','name':'nvidia_net_02_adam'},
      {'get_model':inception, 'correction':0.2, \
       'preprocessing':preprocessing_resize,'optimizer':'adam','gpu':'0','name':'inception_02_adam'},
     {'get_model':vgg16, 'correction':0.2, \
       'preprocessing':preprocessing_resize,'optimizer':'adam','gpu':'0','name':'vgg16_02_adam'}
     
     
     ]

In [6]:
import os
import pickle

def train_model(get_model, correction, preprocessing, name, optimizer, gpu):
    import os

    os.environ['CUDA_VISIBLE_DEVICES']=str(gpu)
    
    model=get_model()
    model.compile(loss='mse', optimizer=optimizer, metrics=['mean_squared_error'])
    
    os.mkdir('results/logs_%s'%name)
    
    checkpoint = ModelCheckpoint("results/model_%name.h5", monitor='val_mean_squared_error', verbose=1,
                              save_best_only=True, mode='min')
    early_stop = EarlyStopping(monitor='val_mean_squared_error',\
                               min_delta=0.001, patience=3,
                                verbose=1, mode='min')

    history = History()

    tb=TensorBoard(log_dir='results/logs_%s'%name, histogram_freq=0, write_graph=True, write_images=False)

    train_generator = generator(deepcopy(images), dataset=0, batch_size=128, preprocessing=preprocessing)
    validation_generator = generator(deepcopy(images), dataset=1, batch_size=128, preprocessing=preprocessing)
    test_generator = generator(deepcopy(images), dataset=2, batch_size=128, preprocessing=preprocessing)

    model.fit_generator(train_generator, \
                    samples_per_epoch=len(images[images.dataset==0]),\
                    nb_epoch=100,\
                    verbose=1,\
                    validation_data=validation_generator, \
                    nb_val_samples=len(images[images.dataset==1]),\
                    callbacks=[checkpoint, early_stop,history,tb]\
                   )
    
    f=open('results/history_%name.pk1','wb')
    pickle.dump(f,history.history,-1)
    f.close()
    
    #realy it's from last iteration - better to load best model here
    res=model.evaluate_generator(test_generator,val_samples=len(images[images.dataset==1]))[0]
    return res

In [7]:
train_model(**grid[0])

ValueError: Negative dimension size caused by subtracting 5 from 3 for 'Conv2D_2' (op: 'Conv2D') with input shapes: [?,3,17,36], [5,5,36,48].

In [12]:
%%sh 

mkdir results